In [1]:
# Import required modules
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import time
import gspread
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials

from datetime import date, datetime, timedelta

from sqlalchemy import create_engine

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-site-isolation-trials")
options.add_argument("start-maximized")

#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)"+"AppleWebKit/537.36 (KHTML, like Gecko)"+"Chrome/87.0.4280.141 Safari/537.36")
options.add_argument(f'user-agent=Chrome/115.0.0.0')
#options.add_argument('--headless')

In [2]:
driver = webdriver.Chrome( options = options)

There was an error managing chrome (request or response body error: operation timed out); using driver found in the cache


In [3]:
driver.get("https://portal.nebosh.org.uk/NeboshWEB/Studying/Find_a_CourseProvider/#")

In [66]:
temp = []

In [75]:
butt = driver.find_elements(By.CSS_SELECTOR,"#ctl00_MainContentPage_ddlQualification option")
for i in butt[23:]:
    i.click()
    time.sleep(2)
    country = driver.find_element(By.CSS_SELECTOR,"#ctl00_MainContentPage_ddlCountries > option:nth-child(2)").click()
    time.sleep(1)
    search = driver.find_element(By.CSS_SELECTOR,"#ctl00_MainContentPage_butSearch").click()
    time.sleep(10)
    while True:
        items = driver.find_elements(By.CSS_SELECTOR, "#accordion > div > a > div > div > h3 > span")
        for item in items:
            item.click()
            time.sleep(1)
            address = driver.find_element(By.CSS_SELECTOR,"#venueDetails > table > tbody > tr > td:nth-child(2)").text
            details = driver.find_elements(By.CSS_SELECTOR,"#providerDetails > table > tbody > tr")
            for det in details:
                if "E-mail" in det.text:
                    email = det.text
                elif "Web" in det.text:
                    web = det.text
                else:
                    pass
            dic = {
                "title":item.text,
                "address":address,
                "Email":email,
                "Web":web     
            }
            temp.append(dic)
        try:
            pagi = driver.find_element(By.CSS_SELECTOR,"#searchPagination > div > a.next").click()
        except:
            break
    driver.find_element(By.CSS_SELECTOR,"#openSearchPanel").click()
    time.sleep(10)
        
        

In [76]:
df = pd.DataFrame(temp)
df

,title,address,Email,Web
0,1. ACT Associates Ltd,"Victoria House, 32 Lower High Street, STOURBRI...",E-mail: central.admin@actassociates.co.uk,Web: www.actassociates.co.uk
1,2. Astutis Ltd,"6 Charnwood Court, Heol Billingsley, Parc Nant...",E-mail: enquiries@astutis.com,Web: www.astutis.com/courses/nebosh-courses
2,3. British Safety Council,"Work Life, Kings House, 174 Hammersmith Road, ...",E-mail: customer.service@britsafe.org,Web: www.britsafe.org
3,4. British Safety Services,"British Safety Services Radio House, Aston Roa...",E-mail: Admin@bssukhse.com,Web: www.bssukhse.com
4,5. BV Associates Ltd,"Terriers House, 201 Amersham Road, HIGH WYCOMB...",E-mail: claire@bvassociates.co.uk,Web: www.bvassociates.co.uk
...,...,...,...,...
952,35. Health and Safety Services,"90 Cornakinnegar Road, Lurgan, Co Armagh, CRAI...",E-mail: info@hassonline.co.uk,Web: www.hassonline.co.uk
953,36. Peak Hse,"Peak HSE Ltd, Unit 8 Twelve O'Clock Court, 21 ...",E-mail: enquiries@peakhse.co.uk,Web: www.peakhse.co.uk
954,37. RDHS Ltd,"32 Highlands Crescent, BOURNEMOUTH, Dorset, BH...",E-mail: hello@rdhs-ltd.co.uk,Web: www.rdhs-ltd.co.uk
955,38. Safety-Zone,"Flat 1 Dunbar's Hospital, 86 Church Street, IN...",E-mail: info@safety-zone.scot,Web: https://www.safety-zone.scot/


In [77]:
df.to_csv("Nebosh course provider.csv",index=False)